In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Load the dataset
data = pd.read_csv('files/iris.csv')

# Encode labels using LabelEncoder
label_encoder = LabelEncoder()
data['Species'] = label_encoder.fit_transform(data['Species'])

# Extract features and labels
X = data[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']].values
y = data['Species'].values
n_classes = len(np.unique(y))  # Number of unique classes


In [2]:
class SVM:
    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000):
        self.lr = learning_rate
        self.lambda_param = lambda_param  # Regularization strength
        self.n_iters = n_iters
        self.w = None
        self.b = None

    def fit(self, X, y):
        # Initialize weights and bias
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        self.b = 0

        # Gradient Descent
        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y[idx] * (np.dot(x_i, self.w) - self.b) >= 1
                if condition:
                    # Apply only the regularization term
                    self.w -= self.lr * (2 * self.lambda_param * self.w)
                else:
                    # Apply hinge loss and regularization term
                    self.w -= self.lr * (2 * self.lambda_param * self.w - np.dot(x_i, y[idx]))
                    self.b -= self.lr * y[idx]

    def predict(self, X):
        # Predict using the decision boundary
        approx = np.dot(X, self.w) - self.b
        return np.sign(approx)


In [3]:
class MultiClassSVM:
    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000):
        self.lr = learning_rate
        self.lambda_param = lambda_param  # Regularization strength
        self.n_iters = n_iters
        self.classifiers = {}  # Dictionary to store one SVM classifier per class

    def fit(self, X, y, n_classes):
        # Train an SVM classifier for each class (One-vs-Rest)
        for c in range(n_classes):
            # Create binary labels: 1 for class `c`, -1 for others
            y_binary = np.where(y == c, 1, -1)
            classifier = SVM(learning_rate=self.lr, lambda_param=self.lambda_param, n_iters=self.n_iters)
            classifier.fit(X, y_binary)  # Train classifier for class `c`
            self.classifiers[c] = classifier  # Store the classifier

    def predict(self, X):
        # Predict for each classifier and select the class with the highest decision score
        scores = np.zeros((X.shape[0], len(self.classifiers)))
        for c, classifier in self.classifiers.items():
            scores[:, c] = np.dot(X, classifier.w) - classifier.b  # Decision score for class `c`
        return np.argmax(scores, axis=1)  # Return the class with the highest score


In [2]:
# Assuming we have loaded and preprocessed the data
from sklearn.model_selection import train_test_split

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
X_test

array([[6.1, 2.8, 4.7, 1.2],
       [5.7, 3.8, 1.7, 0.3],
       [7.7, 2.6, 6.9, 2.3],
       [6. , 2.9, 4.5, 1.5],
       [6.8, 2.8, 4.8, 1.4],
       [5.4, 3.4, 1.5, 0.4],
       [5.6, 2.9, 3.6, 1.3],
       [6.9, 3.1, 5.1, 2.3],
       [6.2, 2.2, 4.5, 1.5],
       [5.8, 2.7, 3.9, 1.2],
       [6.5, 3.2, 5.1, 2. ],
       [4.8, 3. , 1.4, 0.1],
       [5.5, 3.5, 1.3, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.1, 3.8, 1.5, 0.3],
       [6.3, 3.3, 4.7, 1.6],
       [6.5, 3. , 5.8, 2.2],
       [5.6, 2.5, 3.9, 1.1],
       [5.7, 2.8, 4.5, 1.3],
       [6.4, 2.8, 5.6, 2.2],
       [4.7, 3.2, 1.6, 0.2],
       [6.1, 3. , 4.9, 1.8],
       [5. , 3.4, 1.6, 0.4],
       [6.4, 2.8, 5.6, 2.1],
       [7.9, 3.8, 6.4, 2. ],
       [6.7, 3. , 5.2, 2.3],
       [6.7, 2.5, 5.8, 1.8],
       [6.8, 3.2, 5.9, 2.3],
       [4.8, 3. , 1.4, 0.3],
       [4.8, 3.1, 1.6, 0.2]])

In [7]:
# Initialize and train the multiclass SVM model
n_classes = len(np.unique(y))
multi_svm = MultiClassSVM(learning_rate=0.001, lambda_param=0.01, n_iters=1000)
multi_svm.fit(X_train, y_train, n_classes)

# Predict and evaluate
predictions = multi_svm.predict(X_test)
accuracy = np.mean(predictions == y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 86.67%


In [8]:
predictions

array([1, 0, 2, 2, 1, 0, 1, 2, 2, 1, 2, 0, 0, 0, 0, 2, 2, 1, 2, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0], dtype=int64)

In [20]:
new_sample = np.array([[5.5,3.3,4.4,1.1]])
ans = multi_svm.predict(new_sample)
name = label_encoder.inverse_transform(ans)
name[0]

'Iris-versicolor'

1